### CSCN8020-25F-Sec1-Reinforcement Learning Programming
### Assignment one 
#### Name : Haysam Elamin
#### StID : 8953681

## Problem 3 — 5×5 Gridworld 

### MDP Setup
- **States:**  
  $$
  S = \{s_{i,j} \mid i,j \in \{0,1,2,3,4\}\}
  $$
  Each cell in the 5×5 grid is a state, identified by its row and column.

- **Terminal/Goal state:**  
  $$
  s_{\text{goal}} = s_{4,4}
  $$
  Entering this state ends the episode.

- **Grey states (bad states):**  
  $$
  G = \{s_{1,2},\; s_{3,0},\; s_{0,4}\}
  $$

- **Actions:**  
  $$
  A = \{\text{right}, \text{left}, \text{down}, \text{up}\}
  $$

- **Transition model:**  
  If the action is valid, the transition is deterministic:
  $$
  P(s'|s,a) = 1 \quad \text{if } s' \text{ is the neighbor from action } a
  $$
  If invalid (outside the grid), then:
  $$
  s' = s
  $$

- **Rewards:**  
  $$
  R(s) =
  \begin{cases}
  +10 & \text{if } s = s_{4,4} \\
  -5 & \text{if } s \in \{s_{1,2}, s_{3,0}, s_{0,4}\} \\
  -1 & \text{otherwise}
  \end{cases}
  $$

---

### Bellman Optimality Update
The value iteration update rule is:

$$
V_{k+1}(s) =
\begin{cases}
R(s) & \text{if } s \text{ is terminal} \\
\max\limits_{a \in A} \Big[ R(s) + \gamma \cdot V_k(s'(s,a)) \Big], & \text{otherwise}
\end{cases}
$$

The greedy policy is then extracted as:

$$
\pi(s) = \arg\max_{a \in A} \Big[ R(s) + \gamma \cdot V(s'(s,a)) \Big]
$$





In [1]:
"""
Task 1: Define the reward function R(s) for the 5x5 Gridworld.

Rules:
- Goal state (4,4) has reward +10
- Grey states [(1,2), (3,0), (0,4)] have reward -5
- All other states have reward -1
"""

import numpy as np

# ------------------------------
# Step 1: Define grid size
# ------------------------------
n = 5   # This is a 5x5 Gridworld, so we have 25 states in total

# ------------------------------
# Step 2: Define special states
# ------------------------------
goal_state = (4, 4)             # The terminal/goal state (bottom-right corner)
grey_states = [(1, 2), (3, 0), (0, 4)]  # The "bad" states (non-terminal but penalized)

# ------------------------------
# Step 3: Initialize rewards
# ------------------------------
# Start by assigning -1 (step penalty) to ALL states.
# This encourages the agent to reach the goal in the fewest steps possible.
R = -1 * np.ones((n, n))

# ------------------------------
# Step 4: Update reward for goal state
# ------------------------------
# Assign +10 reward to the goal state.
# This makes it attractive for the agent to navigate towards.
R[goal_state] = 10

# ------------------------------
# Step 5: Update rewards for grey states
# ------------------------------
# These are "non-favorable" states that punish the agent with -5 if entered.
for g in grey_states:
    R[g] = -5

# ------------------------------
# Step 6: Display the reward function
# ------------------------------
# This prints the reward grid in matrix form
# where each entry corresponds to the reward of that state.
print("Reward Function R(s) for the 5x5 Gridworld:")
print(R)

# For clarity, let’s also print it in a grid layout with labels
print("\nFormatted Reward Grid (row x col):")
for i in range(n):
    row_repr = []
    for j in range(n):
        if (i, j) == goal_state:
            row_repr.append(" G(+10) ")
        elif (i, j) in grey_states:
            row_repr.append(" X(-5) ")
        else:
            row_repr.append(f"{R[i, j]:>5}")
    print(" | ".join(row_repr))


Reward Function R(s) for the 5x5 Gridworld:
[[-1. -1. -1. -1. -5.]
 [-1. -1. -5. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-5. -1. -1. -1. -1.]
 [-1. -1. -1. -1. 10.]]

Formatted Reward Grid (row x col):
 -1.0 |  -1.0 |  -1.0 |  -1.0 |  X(-5) 
 -1.0 |  -1.0 |  X(-5)  |  -1.0 |  -1.0
 -1.0 |  -1.0 |  -1.0 |  -1.0 |  -1.0
 X(-5)  |  -1.0 |  -1.0 |  -1.0 |  -1.0
 -1.0 |  -1.0 |  -1.0 |  -1.0 |  G(+10) 


In [2]:
from value_iteration import GridWorld
from value_iteration_agent import Agent

env = GridWorld(5)
print("env size:", env.get_size())
print("is terminal(4,4):", env.is_terminal_state(4,4))

agent = Agent(env, theta_threshold=0.05)
print("Agent created. V shape:", agent.get_value_function().shape)

# probe one Bellman update at a non-terminal state:
try:
    v, a, _ = agent.calculate_max_value(0,0)
    print("Agent.calculate_max_value(0,0) ->", v, a)
except Exception as e:
    print("Agent.calculate_max_value error:", repr(e))


env size: 5
is terminal(4,4): True
Agent created. V shape: (5, 5)
Agent.calculate_max_value(0,0) -> -1.0 0


#### 1 — Import, configure, and run Value Iteration

In [3]:
# --- Part 2: Run Value Iteration and obtain V* and π* ---

import numpy as np


from value_iteration import GridWorld
from value_iteration_agent import Agent

# -----------------------------
# 1) Configure environment/agent
# -----------------------------
ENV_SIZE = 5
THETA_THRESHOLD = 0.05     # convergence tolerance; lower = more precise, more iters
MAX_ITERATIONS = 1000

env = GridWorld(ENV_SIZE)
agent = Agent(env, THETA_THRESHOLD)

# -----------------------------
# 2) Value Iteration loop 
# -----------------------------
done = False
for it in range(MAX_ITERATIONS):
    if done:
        break
    V_old = agent.get_value_function()
    new_V = np.copy(V_old)

    # Bellman optimality update for every non-terminal state
    for i in range(ENV_SIZE):
        for j in range(ENV_SIZE):
            if not env.is_terminal_state(i, j):
                # calculate_max_value returns (best_value, best_action, q_values) 
                best_v, _, _ = agent.calculate_max_value(i, j)
                new_V[i, j] = best_v

    # Convergence check: Δ = max |V_new - V_old|
    done = agent.is_done(new_V)
    agent.update_value_function(new_V)

print(f"Converged in {it+1} iteration(s) (or hit cap {MAX_ITERATIONS}).")
V_star = agent.get_value_function()

# Derive the greedy policy π* from V*
agent.update_greedy_policy()
print("V* (numpy array):\n", V_star)
print("\nGreedy policy π*:")
agent.print_policy()


Converged in 10 iteration(s) (or hit cap 1000).
V* (numpy array):
 [[ 0.95011888  2.05275672  3.21342812  4.4351875   5.72125   ]
 [ 2.05275672  3.21342812  4.4351875   5.72125     7.075     ]
 [ 3.21342812  4.4351875   5.72125     7.075       8.5       ]
 [ 4.4351875   5.72125     7.075       8.5        10.        ]
 [ 5.72125     7.075       8.5        10.          0.        ]]

Greedy policy π*:
['Right|Down', 'Right|Down', 'Right', 'Down', 'Down']
['Right|Down', 'Down', 'Right|Down', 'Right|Down', 'Down']
['Right', 'Right|Down', 'Right|Down', 'Right|Down', 'Down']
['Right|Down', 'Right|Down', 'Right|Down', 'Right|Down', 'Down']
['Right', 'Right', 'Right', 'Right', 'X']


#### figures: 5×5 tables for 


In [4]:
import numpy as np

# Helper: print a Markdown table from a 2D list of strings
def print_markdown_table(title, rows):
    print(f"### {title}\n")
    n = len(rows[0])
    header = "|" + "|".join([f" c{j} " for j in range(n)]) + "|"
    sep    = "|" + "|".join(["---" for _ in range(n)]) + "|"
    print(header)
    print(sep)
    for r in rows:
        print("|" + "|".join(r) + "|")
    print()

# 1) Values table (rounded)
vals_str = [[f"{V_star[i,j]:6.2f}" for j in range(ENV_SIZE)] for i in range(ENV_SIZE)]
print_markdown_table("Figure: Optimal State-Values V*", vals_str)

# 2) Policy table (arrows)
# Many class templates store the greedy policy inside the agent; if not, adapt below.
# Try to use agent.policy_grid if your Agent exposes it; otherwise, build from print_policy again.
try:
    pol = agent.policy_grid  # e.g., a 2D array of arrow strings like "↑", "→", "↑→" for ties
except AttributeError:
    # Fallback: derive a 2D policy grid from agent (common pattern)
    # If your Agent has a method to return the greedy action per state, use it instead.
    pol = np.full_like(V_star, "", dtype=object)
    arrows = agent.get_greedy_policy_arrows() if hasattr(agent, "get_greedy_policy_arrows") else None
    if arrows is not None:
        pol = arrows
    else:
        # As a last resort, parse agent.print_policy() output is cumbersome;
        # better to expose a policy grid in your Agent class.
        print("Note: Expose a policy grid in Agent (e.g., agent.policy_grid) for a clean table.")

# If we have pol as a 2D array of strings, print it nicely:
if isinstance(pol, np.ndarray) and pol.shape == V_star.shape:
    pol_str = [[f"{pol[i,j]:^6}" for j in range(ENV_SIZE)] for i in range(ENV_SIZE)]
    print_markdown_table("Figure: Optimal Policy π* (arrows)", pol_str)


### Figure: Optimal State-Values V*

| c0 | c1 | c2 | c3 | c4 |
|---|---|---|---|---|
|  0.95|  2.05|  3.21|  4.44|  5.72|
|  2.05|  3.21|  4.44|  5.72|  7.07|
|  3.21|  4.44|  5.72|  7.07|  8.50|
|  4.44|  5.72|  7.07|  8.50| 10.00|
|  5.72|  7.07|  8.50| 10.00|  0.00|

### Figure: Optimal Policy π* (arrows)

| c0 | c1 | c2 | c3 | c4 |
|---|---|---|---|---|
|      |      |      |      |      |
|      |      |      |      |      |
|      |      |      |      |      |
|      |      |      |      |      |
|      |      |      |      |      |



In [5]:
import numpy as np
import random
from collections import defaultdict

class OffPolicyMonteCarloAgent:
    def __init__(self, env, gamma=0.9, episodes=5000):
        """
        Off-policy Monte Carlo agent with Importance Sampling.

        Parameters
        ----------
        env : GridWorld object
            The environment (same as in Problem 3).
        gamma : float
            Discount factor (default = 0.9).
        episodes : int
            Number of episodes to run for training.
        """
        self.env = env
        self.gamma = gamma
        self.episodes = episodes

        # --- State-value function V(s) ---
        # We keep estimates of value for each state (initialized to 0).
        self.V = np.zeros((env.get_size(), env.get_size()))

        # --- Cumulative weights C(s) ---
        # Used for weighted importance sampling updates.
        self.C = np.zeros((env.get_size(), env.get_size()))

        # --- Target policy (π) ---
        # We’ll store the greedy policy as action indices (0=→,1=←,2=↓,3=↑, -1=Goal).
        self.policy = np.full((env.get_size(), env.get_size()), -1, dtype=int)

    # -------------------------------------------------------------
    # Behavior Policy b(a|s): Random uniform over 4 actions
    # -------------------------------------------------------------
    def behavior_policy(self, state):
        """Choose an action uniformly at random from {Right, Left, Down, Up}."""
        return np.random.choice(len(self.env.get_actions()))

    # -------------------------------------------------------------
    # Generate one episode following the behavior policy
    # -------------------------------------------------------------
    def generate_episode(self):
        """
        Generate a single episode:
        - Start from a random non-terminal state
        - Follow the behavior policy until reaching the goal (terminal)
        Returns:
            episode : list of (state, action, reward)
        """
        env_size = self.env.get_size()
        
        # Pick a random starting position (not terminal)
        i, j = np.random.randint(0, env_size), np.random.randint(0, env_size)
        while self.env.is_terminal_state(i, j):
            i, j = np.random.randint(0, env_size), np.random.randint(0, env_size)
        
        episode = []
        done = False
        
        # Generate the episode
        while not done:
            a = self.behavior_policy((i, j))   # pick action from behavior policy
            ni, nj, r, done = self.env.step(a, i, j)  # take step in env
            episode.append(((i, j), a, r))     # record transition
            i, j = ni, nj                      # move to next state
        
        return episode

    # -------------------------------------------------------------
    # Off-Policy Monte Carlo with Importance Sampling
    # -------------------------------------------------------------
    def update(self):
        """
        Run Monte Carlo updates for multiple episodes.
        Uses Weighted Importance Sampling to update V(s).
        """
        for ep in range(self.episodes):
            episode = self.generate_episode()
            G = 0.0     # return accumulator
            W = 1.0     # importance sampling weight

            # Process episode backwards (MC update)
            for t in reversed(range(len(episode))):
                (s_i, s_j), a, r = episode[t]
                
                # Update return
                G = self.gamma * G + r

                # Update cumulative weight for this state
                self.C[s_i, s_j] += W

                # Update V(s) with weighted average:
                # V(s) <- V(s) + (W / C(s)) * (G - V(s))
                self.V[s_i, s_j] += (W / self.C[s_i, s_j]) * (G - self.V[s_i, s_j])
                
                # Compute greedy action under current env dynamics (target policy π)
                best_val, best_a, _ = self.env.calculate_max_value(s_i, s_j)

                # If the action taken in the episode != greedy target policy → weight goes to 0
                if a != best_a:
                    break
                
                # Otherwise, multiply weight by 1/b(a|s).
                # Since behavior policy is uniform random, b(a|s)=0.25 → multiply by 4.
                W *= 1.0 / 0.25  

        # After training, build the greedy target policy
        for i in range(self.env.get_size()):
            for j in range(self.env.get_size()):
                if not self.env.is_terminal_state(i, j):
                    _, best_a, _ = self.env.calculate_max_value(i, j)
                    self.policy[i, j] = best_a

    # -------------------------------------------------------------
    # Pretty-printing helpers
    # -------------------------------------------------------------
    def print_value_function(self):
        """Print rounded state-value function table."""
        print("\nEstimated Value Function (Off-policy MC with IS):")
        print(np.round(self.V, 2))

    def print_policy(self):
        """Print greedy policy arrows."""
        arrow = {0: "→", 1: "←", 2: "↓", 3: "↑", -1: "G"}
        print("\nGreedy Policy (π):")
        for i in range(self.env.get_size()):
            print(" ".join(f"{arrow[self.policy[i,j]]:>2}" for j in range(self.env.get_size())))


In [6]:
from value_iteration import GridWorld

# Create environment
env = GridWorld(env_size=5, gamma=0.95)

# Create Off-policy Monte Carlo agent
mc_agent = OffPolicyMonteCarloAgent(env, gamma=0.9, episodes=10000)

# Train agent
mc_agent.update()

# Print results
mc_agent.print_value_function()
mc_agent.print_policy()

# For comparison, also show Value Iteration results
print("\nValue Function from Value Iteration (Problem 3):")
print(np.round(env.get_value_function(), 2))
env.update_greedy_policy()
env.print_policy()



Estimated Value Function (Off-policy MC with IS):
[[ 0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.  ]
 [ 3.12  4.58  6.2   8.   10.  ]
 [ 3.66  4.65  6.2   8.04  0.  ]]

Greedy Policy (π):
 →  →  →  →  →
 →  →  →  →  →
 →  →  →  →  →
 →  →  →  →  →
 →  →  →  →  G

Value Function from Value Iteration (Problem 3):
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
 →  →  →  →  →
 →  →  →  →  →
 →  →  →  →  →
 →  →  →  →  →
 →  →  →  →  G
